# Single-View Geometry (Python)

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- You will need [GUI backend](https://matplotlib.org/faq/usage_faq.html#what-is-a-backend) to enable interactive plots in `matplotlib`.
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [90]:
%matplotlib tk
import matplotlib.pyplot as plt
import numpy as np
from sympy import *
from sympy import Symbol
from sympy.solvers import solve
import cv2

from PIL import Image

# Provided functions

In [91]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.imshow(im)
    print('Set at least %d lines to compute vanishing point' % min_lines)
    while True:
        print('Click the two endpoints, use the right key to undo, and use the middle key to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print('Need at least %d lines, you have %d now' % (min_lines, n))
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1

    return n, lines, centers

In [92]:
def plot_lines_and_vp(im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10

    plt.figure()
    plt.imshow(im)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')

    plt.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')
    plt.show()

In [93]:
def get_top_and_bottom_coordinates(im, obj):
    """
    For a specific object, prompts user to record the top coordinate and the bottom coordinate in the image.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        obj: string, object name
    Returns:
        coord: np.ndarray of shape (3, 2)
            where coord[:, 0] is the homogeneous coordinate of the top of the object and coord[:, 1] is the homogeneous
            coordinate of the bottom
    """
    plt.figure()
    plt.imshow(im)

    print('Click on the top coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x1, y1 = clicked[0]
    # Uncomment this line to enable a vertical line to help align the two coordinates
    # plt.plot([x1, x1], [0, im.shape[0]], 'b')
    print('Click on the bottom coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x2, y2 = clicked[0]

    plt.plot([x1, x2], [y1, y2], 'b')

    return np.array([[x1, x2], [y1, y2], [1, 1]])

# Your implementation

In [94]:
def get_vanishing_point(lines):
    """
    Solves for the vanishing point using the user-input lines.
    """
    # <YOUR IMPLEMENTATION>
    vp1 = np.cross(lines[:,0].T, lines[:,1].T)
    vp2 = np.cross(lines[:,0].T, lines[:,2].T)
    vp3 = np.cross(lines[:,1].T, lines[:,2].T)
    vp = (vp1+vp2+vp3)/3
    vp = vp/vp[-1]

    return vp

In [95]:
def get_horizon_line(vpts):
    """
    Calculates the ground horizon line.
    """
    # <YOUR IMPLEMENTATION>
    vp1 = vpts[:, 0]
    vp2 = vpts[:, 1]
    
    # thee line formed by twopoints pi andpj is given by their crossproduct
    
    line = np.cross(vp1, vp2)
    
    # Normalize the parameters 
    norm = np.sqrt(line[0]**2 + line[1]**2)
    line = line / norm
    
    return line

In [96]:
def plot_horizon_line(im, line):
    """
    Plots the horizon line.
    """
    # <YOUR IMPLEMENTATION>
    width = im.shape[1]
    x = range(width)
    # line parameters
    y = (- line[2] - line[0] * x) / line[1]
    
    plt.figure()
    plt.imshow(im)
    plt.plot(x, y, 'b', label = 'Horizon line')
    plt.legend()
    plt.show()

In [97]:
def get_camera_parameters(vpts):
    """
    Computes the camera parameters. Hint: The SymPy package is suitable for this.
    """
    # <YOUR IMPLEMENTATION>
    p1 = vpts[:, 0].reshape(-1,1)
    p2 = vpts[:, 1].reshape(-1,1)
    p3 = vpts[:, 2].reshape(-1,1)
    
    # work process
    px = Symbol('px')
    py = Symbol('py')
    focal = Symbol('focal')
    
    # K martix
    K = Matrix([[focal, 0, px], [0, focal, py], [0, 0, 1]])
    
    eq1 = p1.T * (K.inv().T) * K.inv() * p2
    eq2 = p1.T * (K.inv()).T * K.inv() * p3
    eq3 = p2.T * (K.inv()).T * K.inv() * p3

    answer = solve([eq1[0], eq2[0], eq3[0]], (focal, px, py))
    focal, px, py = answer[0]

    return abs(focal), px, py
    

In [98]:
def get_rotation_matrix(vpts, f, u, v):
    """
    Computes the rotation matrix using the camera parameters.
    """
    # <YOUR IMPLEMENTATION>
    z = vpts[:, 0].reshape(-1,3)
    x = vpts[:, 1].reshape(-1, 3)
    y = vpts[:, 2].reshape(-1,3)
    xyz = np.vstack((np.vstack((x,y)),z))
    
    K = np.matrix([[f, 0, u], [0, f, v], [0, 0, 1]], dtype='float')
    
    K_inv = np.linalg.inv(K)
    
    R = np.dot(K_inv, xyz)
    # constraint
    for i in range(R.shape[0]):
        R[:, i] = R[:, i] / np.linalg.norm(R[:, i])
        
    return R

In [112]:
def estimate_height(im, coords_ref, coords_obj, line, vpts):
    """
    Estimates height for a specific object using the recorded coordinates. You might need to plot additional images here for
    your report.
    """
    # <YOUR IMPLEMENTATION>
    h = 1.6764 # height of that person
#     h = 1.8288
    vp = vpts[:, 2] # vertical
    
    t0 = coords_ref[:, 0]
    b0 = coords_ref[:, 1]
    
    r = coords_obj[:, 0]
    b = coords_obj[:, 1]
    
    line_bb0 = np.cross(b0, b)
    v = np.cross(line_bb0, line)
    v = v/ v[-1]
    
    line_vt0 = np.cross(v, t0)
    line_rb = np.cross(r, b)
    t = np.cross(line_vt0, line_rb)
    t = t/ t[-1]
    
    height = h*(np.linalg.norm(r - b)* np.linalg.norm(vp - t)) /(np.linalg.norm(t - b)* np.linalg.norm(vp - r)) 
    
    #Plot 
    plt.figure()
    plt.imshow(im)
    plt.plot([t0[0], b0[0]], [t0[1], b0[1]], 'g', label='Ref line' )
    plt.plot([b[0], b0[0]], [b[1], b0[1]], 'm', label= 'Bottom line')
    plt.plot([t0[0], t[0]], [t0[1], t[1]], 'y', label = 'Top line')
    plt.plot([r[0], b[0]], [r[1], b[1]], 'b', label ='Estimated object line' )
    
    x = range(im.shape[1])
    y = (- line[2] - line[0] * x) / line[1]
    plt.plot(x, y, 'r', label = 'Vanishing line')   
    
    plt.plot([b[0], v[0]], [b[1], v[1]], 'c', label = 'obejct to horizon bottom')
    plt.plot([t[0], v[0]], [t[1], v[1]], 'k', label = 'obejct to horizon top')

    plt.legend()
    
    plt.show()
    
    return height
    

# Main function

In [100]:
# img = np.asarray(Image.open('CSL.jpg'))
im = cv2.imread('CSL.jpg')
b,g,r = cv2.split(im)
im = cv2.merge([r,g,b])

In [101]:
# Part 1
# Get vanishing points for each of the directions
num_vpts = 3
vpts = np.zeros((3, num_vpts))
for i in range(num_vpts):
    print('Getting vanishing point %d' % i)
    # Get at least three lines from user input
    n, lines, centers = get_input_lines(im)
    # <YOUR IMPLEMENTATION> Solve for vanishing point
    vpts[:, i] = get_vanishing_point(lines)
    # Plot the lines and the vanishing point
    plot_lines_and_vp(im, lines, vpts[:, i])

Getting vanishing point 0
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Getting vanishing point 1
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Getting vanishing point 2
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the mi

E:\Anacondaa\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
E:\Anacondaa\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide
E:\Anacondaa\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in true_divide
E:\Anacondaa\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide


In [102]:
print(vpts)

[[-4.11195763e+02  1.25913011e+03  5.11391341e+02]
 [ 2.16634189e+02  2.18410961e+02  7.65053922e+03]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00]]


In [111]:
# <YOUR IMPLEMENTATION> Get the ground horizon line
horizon_line = get_horizon_line(vpts)
# <YOUR IMPLEMENTATION> Plot the ground horizon line
plot_horizon_line(im, horizon_line)

In [104]:
print(horizon_line)

[-1.06372711e-03  9.99999434e-01 -2.17071466e+02]


In [105]:

# Part 2
# <YOUR IMPLEMENTATION> Solve for the camera parameters (f, u, v)
f, u, v = get_camera_parameters(vpts)
print(f,u,v)

824.519351935074 519.199420650109 310.240289644354


In [106]:
# Part 3
# <YOUR IMPLEMENTATION> Solve for the rotation matrix
R = get_rotation_matrix(vpts, f, u, v)
print(R)

[[ 0.50976938 -0.51209818 -0.50716382]
 [ 0.30403346 -0.27169217 -0.30265469]
 [-0.80479738  0.81482441  0.80696034]]


In [107]:
# Part 4
# Record image coordinates for each object and store in map
# img = cv2.imread('CSL.jpg')
objects = ('person', 'CSL building', 'the spike statue', 'the lamp posts')
coords = dict()
for obj in objects:
    coords[obj] = get_top_and_bottom_coordinates(im, obj)

Click on the top coordinate of person
Click on the bottom coordinate of person
Click on the top coordinate of CSL building
Click on the bottom coordinate of CSL building
Click on the top coordinate of the spike statue
Click on the bottom coordinate of the spike statue
Click on the top coordinate of the lamp posts
Click on the bottom coordinate of the lamp posts


In [113]:
# <YOUR IMPLEMENTATION> Estimate heights
for obj in objects[1:]:
    print('Estimating height of %s' % obj)
    height = estimate_height(im, coords['person'], coords[obj], horizon_line, vpts)
    print(height)

Estimating height of CSL building
31.05918487545983
Estimating height of the spike statue
11.71500395144867
Estimating height of the lamp posts
6.540100524274529
